In [1]:
## main libraries
import easse
#from lens import download_model, LENS
## others
#from evaluate import load
from collections import OrderedDict
from pprint import pprint
import Levenshtein
#import argilla as rg
from datasets import load_from_disk
import csv
import datetime
import pandas as pd
from datasets import Dataset
import os

In [10]:
import easse.fkgl

with open("all_og_sentences.txt", "r") as file:
    all_og_sentences = file.read().splitlines()

og_fkgl_scores = []
for i, sentence in enumerate(all_og_sentences):
    scorer = easse.fkgl.FKGLScorer()
    scorer.add(easse.utils.preprocessing.normalize(sentence, tokenizer="intl"))
    og_fkgl_scores.append(scorer.score())  

with open("all_og_sent_fkgl_scores.txt", "w") as file2:
    for score in og_fkgl_scores:
        file2.write(str(score) + "\n")

In [4]:
# Use GPU for evaluation, if it is available
import torch
DEVICES = [0] if torch.cuda.is_available() else None

# Quiet PyTorch Lightning logger
import logging, warnings
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

os.environ['TQDM_DISABLE'] = '1'        # reduce verbosity of lens

In [4]:
# testing LENS as per the lens-demo

complex = [
    "They are culturally akin to the coastal peoples of Papua New Guinea."
]
simple = [
    "They are culturally similar to the people of Papua New Guinea."
]
references = [[
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast."
]]



complex2 = [
    "They are culturally akin to the coastal peoples of Papua New Guinea."
]
simple2 = [
    "They are culturally similar to the people of Papua New Guinea."
]
references2 = [[
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
    "They are culturally similar to the coastal peoples of Papua New Guinea.",
    "They are similar to the Papua New Guinea people living on the coast.",
]]



In [5]:
# Load LENS
lens_path = download_model("davidheineman/lens")
lens = LENS(lens_path, rescale=True)

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 4814.12it/s]


Using LENS with topk=3


In [6]:
# Evaluate the simplification with LENS

scores = lens.score(complex2, simple2, references2, batch_size=16, devices=DEVICES)

print(f'LENS score: {scores}')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 2/2 [00:23<00:00, 11.99s/it]


LENS score: [78.63447914169828]


In [5]:
## MY ACTUAL DATA

In [3]:
languages = ['eng', 'ger', 'fre']
models = ['LLAMA3', 'ChatGPT']
source_datasets = ['EW-SEW', 'DEPlain', 'WiViCo2']
source_dfs = {'eng': None, 'fre': None, 'ger': None}

simplification_files = {
    "eng": [
        "../data_output/english/out_eng_EW-SEW_ChatGPT.csv",
        "../data_output/english/out_eng_EW-SEW_LLAMA3.csv"
    ],
    "fre": [
        "../data_output/french/out_fre_WiViCo2_ChatGPT.csv",
        "../data_output/french/out_fre_WiViCo2_LLAMA3.csv"
    ],
    "ger": [
        "../data_output/german/out_ger_DEPlain_ChatGPT.csv",
        "../data_output/german/out_ger_DEPlain_LLAMA3.csv"
    ]
}


In [7]:
def parse_file_type_2(input_file, prompt_number=1):

    my_dict = OrderedDict()
    
    if input_file.endswith("txt"):
        print(input_file, "will be passed")
        pass

    else:
        
        my_dict = OrderedDict()
        
        with open(input_file, 'r', encoding='latin1') as file:
            reader = csv.reader(file)
            sentence_number = 1
            current_dict = {}
            prompt = "prompt"+str(prompt_number)
    
            next(reader)  # Skip the first row
            for row in reader:
                if not row[0] :
                    continue
                current_dict["source_index"] = row[0]
                current_dict["input"] = row[1]
                current_dict["output"] = row[prompt_number+1]
                current_dict["prompt"] = row[-1]
                
    
                # calculate sequence matching ratio to find sentence pairs with no or just trivial changes
                current_dict['Lev_distance'] = Levenshtein.distance(current_dict["input"], current_dict["output"])
    
    
                my_dict[sentence_number] = current_dict
                sentence_number += 1
                current_dict = {}

        return my_dict




In [8]:
argilla_df = pd.read_json('argilla_eval_dataset24-10-08_17h.json')
argilla_df

,language,model,model_number,source_dataset,id,original_sentence,modified_sentence,source_index,metadata,suggestions,status,_server_id,user_id,fluency_rating,adequacy_rating,simplicity_rating,contains_wrong_information
0,eng,LLAMA3,Model1,EW-SEW,e418ccc0-f793-4080-963d-99213295efb3,"After high school, Carroll attended junior col...",Carroll attended junior college and then trans...,76021,{},{},pending,3f7c2371-d107-4526-8b57-86480d9a2ccb,b06a10e7-e375-4f04-8aac-59295d38d09c,10,10,10,No. (Everything's okay)
1,eng,LLAMA3,Model1,EW-SEW,640697a4-204f-445a-b8b7-7ed5e4e4fcc0,Many spaniel breeds were developed during the ...,Many spaniel breeds were developed in the 19th...,9252,{},{},pending,39e95278-f0b9-48ea-a965-c9081a8295bb,b06a10e7-e375-4f04-8aac-59295d38d09c,10,9,10,No. (Everything's okay)
2,eng,LLAMA3,Model1,EW-SEW,c4e9e0cb-df73-4cf5-9b71-453a26e61604,The song peaked at number six on the Billboard...,The song peaked at number six on the Billboard...,75303,{},{},pending,3b590faa-c32f-40b5-9582-18a7d3f7b90b,b06a10e7-e375-4f04-8aac-59295d38d09c,10,9,9,No. (Everything's okay)
3,eng,LLAMA3,Model1,EW-SEW,bc3f564d-6718-4a1d-80cc-87cb9fdf5889,Over the course of the following decade their ...,They developed a collaborative working relatio...,25814,{},{},pending,f98eaf9f-0849-4a5d-98d7-1978c934dcf5,b06a10e7-e375-4f04-8aac-59295d38d09c,10,10,10,No. (Everything's okay)
4,eng,LLAMA3,Model1,EW-SEW,97b66dfe-eaa2-43ac-9a62-d9b1d203e010,"In the United States, the sawmill was introduc...",The sawmill was introduced to the United State...,79337,{},{},pending,d9cc3c03-e8a1-4d38-9707-f33a0042fdd4,b06a10e7-e375-4f04-8aac-59295d38d09c,10,9,10,No. (Everything's okay)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,fre,ChatGPT,Model2,EW-SEW,5bd736ee-2334-42b3-b69b-212965f7646f,Naver est également constamment appelé «Google...,"Naver est souvent surnommé le ""Google de la Co...",36250,{},{},pending,06093ae4-963f-465e-abd2-1fe4b2f44236,b4f0f6f8-d94f-4738-bfa9-643c4636b565,7,9,8,None
725,fre,ChatGPT,Model2,EW-SEW,a785fd7b-5170-4582-9ba4-973ea43f8291,Il fait suite au jeu Peggle sorti en 2007.,"Ce jeu est la suite de Peggle, sorti en 2007.",13480,{},{},pending,a077935b-0b9a-4693-9280-1b05b97660dc,b4f0f6f8-d94f-4738-bfa9-643c4636b565,8,10,7,None
726,fre,ChatGPT,Model2,EW-SEW,376db04c-3a17-401a-bb38-5521447f9ae1,Power Rangers est une franchise de médias amér...,Power Rangers est une série de super-héros amé...,38375,{},{},pending,96f5e48f-2654-4118-b778-1930fb4190a2,b4f0f6f8-d94f-4738-bfa9-643c4636b565,8,6,8,None
727,fre,ChatGPT,Model2,EW-SEW,ed2f34f6-2862-4add-bdaa-a0e338658e67,"Ils étaient initialement cinq, avec Smaïn et S...","Le groupe initial comprenait cinq membres, don...",13944,{},{},pending,9f6cfaff-ba83-4487-a8d1-7079c1c33cc3,b4f0f6f8-d94f-4738-bfa9-643c4636b565,7,10,7,None


In [9]:
### get source datasets

# English

original_sentences_eng_file = '../data_orig/eng/normal.aligned'
simple_sentences_eng_file = '../data_orig/eng/simple.aligned'

original_sents_eng_df = pd.read_csv(original_sentences_eng_file, sep='\t', header=None, names=['unimportant_string_1', 'unimportant_number_1', 'original_sentence'],  encoding='latin1')

# Read the second TSV file
simple_sents_eng_df = pd.read_csv(simple_sentences_eng_file, sep='\t', header=None, names=['unimportant_string_2', 'unimportant_number_2', 'simple_sentence'],  encoding='latin1')

# Extract the relevant columns
original_column = original_sents_eng_df[['original_sentence']]
simple_column = simple_sents_eng_df[['simple_sentence']]

# Combine the DataFrames into one
result_df = pd.concat([original_column, simple_column], axis=1)

source_eng_df = result_df
source_eng_df.rename(columns={'simple_sentence': 'ref_simplifications'}, inplace=True)

source_dfs['eng'] = source_eng_df

source_eng_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167689 entries, 0 to 167688
Data columns (total 2 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   original_sentence    167689 non-null  object
 1   ref_simplifications  167689 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [10]:
# French

source_file_french = '../data_orig/fre/wivico_dataset_v2.tsv'
source_fre_df = pd.read_csv(source_file_french, sep='\t', header=None, names=['wiki_sent', 'viki_sent', 'wiki', 'viki', 'pair'], skiprows=[0])
source_fre_df.rename(columns={'wiki_sent': 'original_sentence', 'viki_sent': 'ref_simplifications'}, inplace=True)

source_dfs['fre'] = source_fre_df

source_fre_df

,original_sentence,ref_simplifications,wiki,viki,pair
0,Catharanthus roseus La Pervenche de Madagascar...,"La pervenche de Madagascar (nom commun), ou Ca...",0,0,0
1,"Claude de France (Romorantin, 13 octobre 1499 ...",Claude de France est née le 13 octobre 1499 à ...,0,1,0
2,"Hippocrate de Kos Hippocrate de Kos, ou simple...",Hippocrate de Cos (surnommé Hippocrate le Gran...,0,0,0
3,"‌L'ASM Clermont Auvergne, anciennement Associa...","L'ASM Clermont Auvergne, anciennement AS Montf...",0,0,0
4,"‌L'ASM Clermont Auvergne, anciennement Associa...","L'ASM Clermont Auvergne, anciennement AS Montf...",0,0,0
...,...,...,...,...,...
46520,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...","Өвөр Монгол, Öwör Mongol ; en chinois : 内蒙古) e...",0,0,0
46521,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...",La Région autonome de Mongolie-Intérieure ou s...,0,0,0
46522,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...",La Région autonome de Mongolie-Intérieure ou s...,0,0,0
46523,"Өвөр Монгол, Öwör Mongol ; chinois simplifié :...",La Région autonome de Mongolie-Intérieure ou s...,0,0,0


In [11]:
# German
source_file = '../data_orig/ger/all.csv'
source_ger_df = pd.read_csv(source_file)
source_ger_df.rename(columns={'original': 'original_sentence', 'simplification': 'ref_simplifications'}, inplace=True)

source_dfs['ger'] = source_ger_df

source_ger_df

,original_sentence,ref_simplifications,original_id,simplification_id,pair_id,domain,corpus,language_level_original,language_level_simple,license,author,simple_url,complex_url,simple_title,complex_title,access_date,rater,alignment
0,"Am Mittwoch hat der Iran bekanntgegeben, dass ...",Der Iran wird teilweise aus dem Atom-Abkommen ...,296-1-1-0,296-0-1-0,296,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
1,"Die Präsidentin der EU-Kommission, Ursula von ...",Brüssel Ursula von der Leyen ist die Präsident...,628-1-1-0,628-0-1-0|628-0-1-1|628-0-1-2,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:3
2,So sollen die Treibhaus-Gase in der EU bis zum...,Bis zum Jahr 2030 soll es in der EU um 55 Proz...,628-1-1-2,628-0-1-4,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
3,In den letzten 29 Jahren schaffte man eine Ver...,"Das ist sehr viel, denn in den letzten 29 Jahr...",628-1-1-3,628-0-1-5,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
4,Für das neue Ziel bleiben weniger als 10 Jahre.,Für das neue Ziel bleiben aber weniger als 10 ...,628-1-1-4,628-0-1-6,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13117,Der Österreicher Adi Hütter ist Trainer von Ei...,Frankfurt am Main/ Mönchengladbach Adi Hütter ...,769-1-15-0,769-0-14-0|769-0-14-1|769-0-14-3|769-0-14-4,769,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,2,1:4
13118,Er wechselt nach dem Saisonende zu Borussia Mö...,Hütter wird aber mit Saison-Ende in der deutsc...,769-1-15-1,769-0-14-5|769-0-14-7,769,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,2,1:2
13119,Hütter unterschreibt einen Vertrag bis zum Jah...,Er unterschreibt dort einen Vertrag bis zum Ja...,769-1-15-3,769-0-14-8,769,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,2,1:1
13120,Für Hütter muss Borussia Mönchengladbach auch ...,Für Hütter muss Mönchengladbach eine hohe Ablö...,769-1-16-0,769-0-15-0,769,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten

In [12]:
user_mapping = {
    "5d42d341-a876-4707-b069-caf8ed9f8f36": "JW",
    "36985659-71b0-4c64-91a6-f921ad3e6ff2": "AS",
    "26957828-9534-49d7-9c4e-c1ec62ddda1c": "Th",
    "b06a10e7-e375-4f04-8aac-59295d38d09c": "real Al",
    "d84eaa03-d240-4abd-b4eb-d2b8c9d5f6eb": "rw-junk",
    "b4f0f6f8-d94f-4738-bfa9-643c4636b565": "gmx-fan(Nae)",
    "b4219e1a-5bc6-4fbc-b000-284a86c14c9b": "AlA26(Kb)",
    "a61ea0ac-4f34-4eae-a55f-ce8d73823634": "X2",
    "Y2": "language-nerd"
    
}

argilla_df['user'] = argilla_df['user_id'].map(user_mapping)

print(argilla_df["fluency_rating"].value_counts())
print(argilla_df["adequacy_rating"].value_counts())
print(argilla_df["simplicity_rating"].value_counts())
print(argilla_df["language"].value_counts())
print(argilla_df["user_id"].value_counts())

10    556
9      71
8      55
7      32
6      13
4       2
Name: fluency_rating, dtype: int64
10    349
9     144
8      99
7      50
6      31
0      26
5      15
4      10
3       3
2       2
Name: adequacy_rating, dtype: int64
5     159
8     134
9     120
7     117
6     111
10     51
4      17
3      11
0       9
Name: simplicity_rating, dtype: int64
ger    609
fre    103
eng     17
Name: language, dtype: int64
36985659-71b0-4c64-91a6-f921ad3e6ff2    200
5d42d341-a876-4707-b069-caf8ed9f8f36    200
26957828-9534-49d7-9c4e-c1ec62ddda1c    159
b4f0f6f8-d94f-4738-bfa9-643c4636b565    100
a61ea0ac-4f34-4eae-a55f-ce8d73823634     44
d84eaa03-d240-4abd-b4eb-d2b8c9d5f6eb     14
b06a10e7-e375-4f04-8aac-59295d38d09c     10
b4219e1a-5bc6-4fbc-b000-284a86c14c9b      2
Name: user_id, dtype: int64


KeyError: 'simple_sentence'

In [ ]:


lens_path = download_model("davidheineman/lens")
lens = LENS(lens_path, rescale=True)


In [10]:
## ONE LOOP TO MERGE THEM ALL

import easse.bleu
import easse.fkgl
import easse.sari


DO_NOT_SAVE = False


N = 'all'
BATCH_SIZE = 8     # default: 8


for language in languages:

    for model in models:  
    
        simpli_files_for_language = simplification_files[language]
        for simpli_file in simpli_files_for_language:
            


            if not model in str(simpli_file):
                continue
                

            
            
    
            for l, source_df in source_dfs.items():
                if l != language:
                    continue
                print(f"Now using simpli file-- {simpli_file}")
                print(f"with language {l} and model {model}")
                
                

                merged_df_filename = f"merged_df_{language}_{model}.csv"

                try:
                    merged_df = pd.read_csv(merged_df_filename)
                    print(f"OK. Found and loaded {merged_df_filename}.")
                except FileNotFoundError:
                    print(f"!!! File {merged_df_filename} not found. Will be recreated from sources.")

                    df = pd.read_csv(simpli_file, index_col=0, encoding='latin1')
                    df.rename(columns={'Column1': 'OG_sent'}, inplace=True)
                    df = df.dropna(how='all')

                    
                    merged_df = pd.DataFrame()
                    filtered_df = df[df.index.isin(source_df.index)]
                    merged_df = pd.merge(filtered_df, source_df, left_on='source_index', right_index=True)
                

                complex = merged_df["original_sentence"].fillna("").tolist()
                simple = merged_df["simple_sent(prompt1)"].fillna("").tolist()
                references = [[ref] for ref in merged_df["ref_simplifications"].fillna("")]       # references needs to be a list of lists


                if 'LENS' not in merged_df or merged_df['LENS'].isna().all():
                    # Lens-scoring sentences
                                                
                    lens_scores = []
                    if (not 'N' in globals()) or (type(N) is str):
                        print("Lens-scoring all sentences... Limit N is not set.")
                        lens_scores = lens.score(complex, simple, references, batch_size=BATCH_SIZE, devices=DEVICES)
                    else:  
                        print("Lens-scoring N=", N, " sentences...")
                        
                        lens_scores = lens.score(complex[:N], simple[:N], references[:N], batch_size=BATCH_SIZE, devices=DEVICES)
                        if len(lens_scores) < len(merged_df):
                            lens_scores.extend([""]*(len(merged_df)-len(lens_scores)))

                    merged_df["LENS"] = lens_scores
                else:
                    print("LENS-score already in data.")



                if 'SARI' not in merged_df or merged_df['SARI'].isna().all():
                    # SARI-scoring sentences
                    
                    references_flattened = [[item[0] for item in references]]

                    sari_scores = []
                    if (not 'N' in globals()) or (type(N) is str):
                        print("SARI-scoring all sentences... Limit N is not set.")
                        for i in range(len(complex)):
                            score = easse.sari.corpus_sari(orig_sents=[complex[i]], sys_sents=[simple[i]], refs_sents=references_flattened[0][i])
                            sari_scores.append(score)
                    else:  
                        print("SARI-scoring N=", N, " sentences...")
                        sari_scores = easse.sari.corpus_sari(complex[:N], simple[:N], references_flattened[:N], batch_size=BATCH_SIZE, devices=DEVICES)
                        if len(sari_scores) < len(merged_df):
                            sari_scores.extend([""]*(len(merged_df)-len(sari_scores)))

                    print(sari_scores)
                    merged_df["SARI"] = sari_scores

                else:
                    print("SARI-score already in data.")
                    

                #if 'FKGL' not in merged_df or merged_df['FKGL'].isna().all():
                if True:
                    # FKGL-scoring sentences

                    fkgl_scores = []
                    
                    if (not 'N' in globals()) or (type(N) is str):
                        simple = simple
                        print("FKGL-scoring all sentences... Limit N is not set.")   
                    else:
                        simple = simple[:N]
                        print("FKGL-scoring N=", N, " sentences...")   
                        
                    for i, sentence in enumerate(simple):
                        scorer = easse.fkgl.FKGLScorer()
                        scorer.add(easse.utils.preprocessing.normalize(sentence, tokenizer="intl"))
                        fkgl_scores.append(scorer.score())                                   

                    print(fkgl_scores)
                    merged_df["FKGL"] = fkgl_scores

                else:
                    print("FKGL-score already in data.")


                if not merged_df.columns[merged_df.columns.str.contains('^BLEU')].any():
                    
                    if ('N' in globals()) and (type(N) == int): 
                        complex = complex[:N]
                        simple = simple[:N]
                        print("BLEU-scoring N=", N, " sentences...")
                    else:
                        print("BLEU-scoring all sentences... Limit N is not set.") 

                bleu_scores = []
                for i, sentence in enumerate(simple):
                    if i == 0:
                        print(sentence)
                        print(references[i])
                    score = easse.bleu.sentence_bleu(sys_sent=sentence, ref_sents=references[i], lowercase=True, tokenizer='intl')
                    bleu_scores.append(score)

                print(bleu_scores)
                merged_df["BLEU"] = bleu_scores

                if not merged_df.columns[merged_df.columns.str.contains('^BERT')].any():

                    iso_language_mapping = {'eng': 'en', 'fre': 'fr', 'ger': 'de'}
                    lang = iso_language_mapping[language]

                    if ('N' in globals()) and (type(N) == int): 
                        complex = complex[:N]
                        simple = simple[:N]
                        print("BERT-scoring for lang=", lang, "using N=", N, " sentences...")
                    else:
                        print("BERT-scoring all sentences for lang=", lang, "... Limit N is not set.")   
           
                    
                    
                    bertscore = load("bertscore")
                    
                    results = bertscore.compute(predictions=simple, references=complex, model_type='distilbert-base-uncased', lang=lang)
                    for key, value in results.items():
                        column_name = "BERTScore_"+key
                        print(column_name, value[:5])
                        merged_df[column_name] = value

                else:
                    print("BERT-score already in data.")

                
                if not DO_NOT_SAVE:
                    print("TEST")
                    merged_df["source_index"] = merged_df["source_index"].astype(int)
                    merged_df.to_csv(merged_df_filename, index=False)
                
                print("Finished processing ", language, model)
                print("______")

print("|||||| ALL DONE ||||||")

merged_df

Now using simpli file-- ../data_output/english/out_eng_EW-SEW_LLAMA3.csv
with language eng and model LLAMA3
OK. Found and loaded merged_df_eng_LLAMA3.csv.
LENS-score already in data.
SARI-score already in data.
FKGL-scoring all sentences... Limit N is not set.
[10.356923076923078, 9.785555555555558, 10.356923076923078, 13.372727272727271, 0.48666666666666814, 5.993333333333336, 6.936363636363637, 9.44923076923077, 7.698823529411765, 4.790909090909089, 0, 0.5166666666666693, 6.780000000000001, 5.884285714285717, 1.7125000000000021, 0.5166666666666693, 1.0311111111111124, 12.62714285714286, 14.911666666666669, 4.790909090909089, 2.4700000000000024, 15.45, 13.073333333333338, 3.653333333333336, 2.4700000000000024, 3.633333333333333, 9.44923076923077, 4.420000000000002, 6.790000000000003, 6.2755555555555596, 7.368571428571428, 8.353333333333339, 8.370000000000001, 5.806666666666668, 5.863636363636363, 2.0600000000000023, 5.863636363636363, 6.010000000000002, 2.342222222222226, 4.8300000000

,source_index,OG_sent,simple_sent(prompt1),simple_sent(prompt2),Prompt1,Prompt2,Model,original_sentence,ref_simplifications,wiki,viki,pair,LENS,SARI,FKGL,BERTScore_precision,BERTScore_recall,BERTScore_f1,BERTScore_hashcode,BLEU
0,37241,Panurus biarmicus La Panure à moustaches (Panu...,"La Panure à moustaches est un oiseau d'Europe,...",NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,Panurus biarmicus La Panure à moustaches (Panu...,La panure à moustaches est un passereau d'Eura...,0,1,0,53.301664,30.519752,3.276190,0.897083,0.829452,0.861943,distilbert-base-uncased_L5_no-idf_version=0.3....,26.608125
1,35958,"Monsieur Wens, surnom de Wenceslas Vorobeïtchi...",Monsieur Wens est un personnage de roman polic...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,"Monsieur Wens, surnom de Wenceslas Vorobeïtchi...",Monsieur Wens est un personnage de fiction cr...,0,1,0,47.054342,16.841231,6.960000,0.981091,0.922961,0.951139,distilbert-base-uncased_L5_no-idf_version=0.3....,22.293103
2,13955,Ils permettent aux enfants de s'amuser en fais...,Les enfants s'amusent en construisant des chât...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,Ils permettent aux enfants de s'amuser en fais...,Il permet aux enfants de s'amuser en faisant d...,0,1,0,53.916683,28.400285,8.333333,0.937786,0.897071,0.916976,distilbert-base-uncased_L5_no-idf_version=0.3....,15.362208
3,39745,Saint-Ignace-de-Loyola est une municipalité du...,Saint-Ignace-de-Loyola est une ville du Québec...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,Saint-Ignace-de-Loyola est une municipalité du...,Saint-Ignace-de-Loyola est une petite municipa...,0,1,0,58.769179,26.271023,5.180000,0.964036,0.911023,0.936780,distilbert-base-uncased_L5_no-idf_version=0.3....,54.478672
4,25619,"Le hanafisme ou hanéfisme (en arabe : ????, ?a...",Le hanafisme est l'une des plus anciennes écol...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,"Le hanafisme ou hanéfisme (en arabe : حنفي, Ḥa...","On parle aussi d'écoles juridiques, car ces éc...",0,1,0,50.990206,32.033778,5.993333,0.914084,0.799095,0.852731,distilbert-base-uncased_L5_no-idf_version=0.3....,0.998085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,42801,Un banc est un groupement important de poisson...,Un banc est un groupe de poissons de la même e...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,Un banc est un groupement important de poisson...,Un banc de poissons est un grand groupe de poi...,0,1,0,48.346710,23.302169,2.060000,0.969729,0.914128,0.941108,distilbert-base-uncased_L5_no-idf_version=0.3....,48.991130
1096,44032,"Un satellite de reconnaissance, ou satellite e...",Un satellite espion est un satellite utilisé p...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,"Un satellite de reconnaissance, ou satellite e...",Un satellite de reconnaissance (satellite es...,0,1,0,62.954711,27.145712,11.500000,0.965764,0.905158,0.934479,distilbert-base-uncased_L5_no-idf_version=0.3....,26.673027
1097,38504,Progressistes (en portugais : Progressistas) ...,Les Progressistes sont un parti politique brés...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,Progressistes (en portugais : Progressistas) —...,Progressistas est un parti politique au Brésil.,0,1,0,70.639613,29.684792,5.818462,0.928146,0.842644,0.883331,distilbert-base-uncased_L5_no-idf_version=0.3....,8.263103
1098,28169,Le pop rock est un genre musical mêlant le sty...,Le pop rock est un genre de musique qui mélang...,NaN,Convertis chacune de ces 50 phrases originales...,NaN,ChatGPT-4o,Le pop rock est un genre musical mêlant le sty...,Le pop rock est un genre musical dérivé du roc...,0,1,0,63.847101,28.692151,0.827143,0.926494,0.858659,0.891288,distilbert-base-uncased_L5_no-idf_version=0.3....,18.793352
